In [1]:
from datetime import datetime # Date calculations
from datetime import timedelta
from dateutil.relativedelta import relativedelta # Exact date calcs
import pandas as pd #DataFrames
import numpy as np #
import math # Round up function
import random # Random loan term generation
from tqdm import tqdm # Loading bars

## Helper Functions

In [14]:
def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return(math.ceil(n * multiplier) / multiplier)

# Return a random datetime between to datetime objects
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    return(start + timedelta(seconds=random_second))

def zero_division(n, d):
    return n / d if d else 0

## Loan Terms

In [58]:
terms = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':0.5,
 'origination_points':0.0,
 'origination_points_deferred':0.0,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d'),
 'amortization_term':30
}

terms_full_ir = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':1,
 'origination_points':0.0,
 'origination_points_deferred':0.0,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d'),
 'amortization_term':30
}

terms_no_am = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':0.5,
 'origination_points':0.0,
 'origination_points_deferred':0.0,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d'),
 'amortization_term':0
}

terms_full_ir_no_am = {'principal':10000000,
 'cash_rate':0.05,
 'interest_reserve_rate':0.05,
 'interest_reserve_term':1,
 'origination_points':0.0,
 'origination_points_deferred':0.0,
 'date_originated':datetime.strptime('2019-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2020-01-01', '%Y-%m-%d'),
 'amortization_term':0
}

terms_no_ir_no_am = {'principal':10000000,
 'cash_rate':0.1,
 'interest_reserve_rate':0.0,
 'interest_reserve_term':1,
 'origination_points':0.0,
 'origination_points_deferred':0.0,
 'date_originated':datetime.strptime('2020-01-01', '%Y-%m-%d'),
 'date_payoff':datetime.strptime('2021-01-01', '%Y-%m-%d'),
 'amortization_term':0
}

def generate_loan_terms():
    principal = random.uniform(1000000,50000000)
    cash_rate = random.uniform(0.05,0.09)
    interest_reserve_rate = random.uniform(0, 0.05)
    interest_reserve_term = random.random()
    origination_points = random.uniform(0, 0.01)
    origination_points_deferred = random.uniform(0, 0.01)
    date_originated = random_date(datetime.strptime('2020-01-01', '%Y-%m-%d'), datetime.strptime('2022-01-01', '%Y-%m-%d'))
    date_payoff = date_originated + timedelta(int(random.uniform(12,24))*365/12)
    terms = {'principal':principal,'cash_rate':cash_rate,'interest_reserve_rate':interest_reserve_rate,'interest_reserve_term':interest_reserve_term,'origination_points':origination_points,'origination_points_deferred':origination_points_deferred,'date_originated':date_originated,'date_payoff':date_payoff}
    return(terms)

In [38]:
generate_loan_terms()

{'cash_rate': 0.0770544951048719,
 'date_originated': datetime.datetime(2020, 6, 25, 17, 48, 40),
 'date_payoff': datetime.datetime(2021, 7, 26, 3, 48, 40),
 'interest_reserve_rate': 0.0089917740117538,
 'interest_reserve_term': 0.0621229123895064,
 'origination_points': 0.0008501623662776148,
 'origination_points_deferred': 0.004857032158255047,
 'principal': 19679139.979377758}

## Loan CFs

In [52]:
def generate_loan(terms):
    '''
    Periodic transactions.
    '''
    # Loan cash flows are calculated on a daily basis. Generate range of dates between origination and payoff date
    # # https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    term = (terms['date_payoff'] - terms['date_originated']).days
    term_ir = round_up(term * terms['interest_reserve_term'],0)
    date_range = pd.date_range(terms['date_originated'] + relativedelta(days = 1), periods = term - 1, freq = 'D')
    date_range_ir = pd.date_range(terms['date_originated'] + relativedelta(days = 1), periods = term_ir - 1, freq = 'D')

    # Payments are calculated on a daily basis, on a 365/360 day convention, rounded to the nearest cent.
    rate_cash = terms['cash_rate'] / 360
    rate_ir = terms['interest_reserve_rate'] / 360

    rate_coupon = rate_cash + rate_ir
    n = ((terms['date_originated'] + relativedelta(years = terms['amortization_term'])) - terms['date_originated']).days

    coupon = (terms['principal'] * rate_coupon) + zero_division((terms['principal'] * rate_coupon),((1 + rate_coupon) ** n - 1))
    
    balance_coupon = coupon * term
    balance_ir = balance_coupon * (rate_ir / rate_coupon) * terms['interest_reserve_term']
    print(balance_ir)
    cash = round_up(balance_coupon / term, 2)
    ir = round_up(balance_ir / term_ir, 2)

    daily_cash = pd.Series(cash, index = date_range)
    daily_ir = pd.Series(ir, index = date_range_ir)

    # Remove interest reserve from cash flow stream
    daily_cash = daily_cash.subtract(daily_ir, fill_value = 0)

    '''
    One-time transactions
    '''
    rate = terms['cash_rate'] + (terms['interest_reserve_rate'] * terms['interest_reserve_term']) + terms['origination_points'] + terms['origination_points_deferred']
    points_in = terms['origination_points'] * terms['principal']
    points_out = terms['origination_points_deferred'] * terms['principal']
    payoff = (terms['principal'] * (1 + rate_coupon) ** term) - coupon * ( ((1 + rate_coupon) ** term - 1) / rate_coupon ) + points_out
    net_funding = -(terms['principal'] - points_in - balance_ir)

    '''
    {
        'net_funding' : [{'2019-01-01' : -1000}],
        'cash' : [{'2019-01-01' : 1}]
    }
    '''

    # Create loan schedule

    net_funding = pd.Series(net_funding, index = pd.Index([terms['date_originated']]))
    payoff = pd.Series(payoff, index = pd.Index([terms['date_payoff']]))

    cash_flows = {
        'profit (loss)' : sum(net_funding) + sum(daily_cash) + sum(payoff),
        'net_funding' : net_funding,
        'cash' : daily_cash,
        'payoff' : payoff
    }

    return(cash_flows)

In [53]:
x = generate_loan(terms)
x

266159.89710142155


{'cash': 2019-01-02    1462.40
 2019-01-03    1462.40
 2019-01-04    1462.40
 2019-01-05    1462.40
 2019-01-06    1462.40
 2019-01-07    1462.40
 2019-01-08    1462.40
 2019-01-09    1462.40
 2019-01-10    1462.40
 2019-01-11    1462.40
 2019-01-12    1462.40
 2019-01-13    1462.40
 2019-01-14    1462.40
 2019-01-15    1462.40
 2019-01-16    1462.40
 2019-01-17    1462.40
 2019-01-18    1462.40
 2019-01-19    1462.40
 2019-01-20    1462.40
 2019-01-21    1462.40
 2019-01-22    1462.40
 2019-01-23    1462.40
 2019-01-24    1462.40
 2019-01-25    1462.40
 2019-01-26    1462.40
 2019-01-27    1462.40
 2019-01-28    1462.40
 2019-01-29    1462.40
 2019-01-30    1462.40
 2019-01-31    1462.40
                ...   
 2019-12-02    2916.83
 2019-12-03    2916.83
 2019-12-04    2916.83
 2019-12-05    2916.83
 2019-12-06    2916.83
 2019-12-07    2916.83
 2019-12-08    2916.83
 2019-12-09    2916.83
 2019-12-10    2916.83
 2019-12-11    2916.83
 2019-12-12    2916.83
 2019-12-13    2916.83
 20

In [54]:
y = generate_loan(terms_full_ir)
y

532319.7942028431


{'cash': 2019-01-02    1458.41
 2019-01-03    1458.41
 2019-01-04    1458.41
 2019-01-05    1458.41
 2019-01-06    1458.41
 2019-01-07    1458.41
 2019-01-08    1458.41
 2019-01-09    1458.41
 2019-01-10    1458.41
 2019-01-11    1458.41
 2019-01-12    1458.41
 2019-01-13    1458.41
 2019-01-14    1458.41
 2019-01-15    1458.41
 2019-01-16    1458.41
 2019-01-17    1458.41
 2019-01-18    1458.41
 2019-01-19    1458.41
 2019-01-20    1458.41
 2019-01-21    1458.41
 2019-01-22    1458.41
 2019-01-23    1458.41
 2019-01-24    1458.41
 2019-01-25    1458.41
 2019-01-26    1458.41
 2019-01-27    1458.41
 2019-01-28    1458.41
 2019-01-29    1458.41
 2019-01-30    1458.41
 2019-01-31    1458.41
                ...   
 2019-12-02    1458.41
 2019-12-03    1458.41
 2019-12-04    1458.41
 2019-12-05    1458.41
 2019-12-06    1458.41
 2019-12-07    1458.41
 2019-12-08    1458.41
 2019-12-09    1458.41
 2019-12-10    1458.41
 2019-12-11    1458.41
 2019-12-12    1458.41
 2019-12-13    1458.41
 20

In [55]:
generate_loan(terms_no_am)

253472.22222222222


{'cash': 2019-01-02    1392.68
 2019-01-03    1392.68
 2019-01-04    1392.68
 2019-01-05    1392.68
 2019-01-06    1392.68
 2019-01-07    1392.68
 2019-01-08    1392.68
 2019-01-09    1392.68
 2019-01-10    1392.68
 2019-01-11    1392.68
 2019-01-12    1392.68
 2019-01-13    1392.68
 2019-01-14    1392.68
 2019-01-15    1392.68
 2019-01-16    1392.68
 2019-01-17    1392.68
 2019-01-18    1392.68
 2019-01-19    1392.68
 2019-01-20    1392.68
 2019-01-21    1392.68
 2019-01-22    1392.68
 2019-01-23    1392.68
 2019-01-24    1392.68
 2019-01-25    1392.68
 2019-01-26    1392.68
 2019-01-27    1392.68
 2019-01-28    1392.68
 2019-01-29    1392.68
 2019-01-30    1392.68
 2019-01-31    1392.68
                ...   
 2019-12-02    2777.78
 2019-12-03    2777.78
 2019-12-04    2777.78
 2019-12-05    2777.78
 2019-12-06    2777.78
 2019-12-07    2777.78
 2019-12-08    2777.78
 2019-12-09    2777.78
 2019-12-10    2777.78
 2019-12-11    2777.78
 2019-12-12    2777.78
 2019-12-13    2777.78
 20

In [56]:
generate_loan(terms_full_ir_no_am)

506944.44444444444


{'cash': 2019-01-02    1388.89
 2019-01-03    1388.89
 2019-01-04    1388.89
 2019-01-05    1388.89
 2019-01-06    1388.89
 2019-01-07    1388.89
 2019-01-08    1388.89
 2019-01-09    1388.89
 2019-01-10    1388.89
 2019-01-11    1388.89
 2019-01-12    1388.89
 2019-01-13    1388.89
 2019-01-14    1388.89
 2019-01-15    1388.89
 2019-01-16    1388.89
 2019-01-17    1388.89
 2019-01-18    1388.89
 2019-01-19    1388.89
 2019-01-20    1388.89
 2019-01-21    1388.89
 2019-01-22    1388.89
 2019-01-23    1388.89
 2019-01-24    1388.89
 2019-01-25    1388.89
 2019-01-26    1388.89
 2019-01-27    1388.89
 2019-01-28    1388.89
 2019-01-29    1388.89
 2019-01-30    1388.89
 2019-01-31    1388.89
                ...   
 2019-12-02    1388.89
 2019-12-03    1388.89
 2019-12-04    1388.89
 2019-12-05    1388.89
 2019-12-06    1388.89
 2019-12-07    1388.89
 2019-12-08    1388.89
 2019-12-09    1388.89
 2019-12-10    1388.89
 2019-12-11    1388.89
 2019-12-12    1388.89
 2019-12-13    1388.89
 20

In [59]:
generate_loan(terms_no_ir_no_am)

0.0


{'cash': 2020-01-02    2777.78
 2020-01-03    2777.78
 2020-01-04    2777.78
 2020-01-05    2777.78
 2020-01-06    2777.78
 2020-01-07    2777.78
 2020-01-08    2777.78
 2020-01-09    2777.78
 2020-01-10    2777.78
 2020-01-11    2777.78
 2020-01-12    2777.78
 2020-01-13    2777.78
 2020-01-14    2777.78
 2020-01-15    2777.78
 2020-01-16    2777.78
 2020-01-17    2777.78
 2020-01-18    2777.78
 2020-01-19    2777.78
 2020-01-20    2777.78
 2020-01-21    2777.78
 2020-01-22    2777.78
 2020-01-23    2777.78
 2020-01-24    2777.78
 2020-01-25    2777.78
 2020-01-26    2777.78
 2020-01-27    2777.78
 2020-01-28    2777.78
 2020-01-29    2777.78
 2020-01-30    2777.78
 2020-01-31    2777.78
                ...   
 2020-12-02    2777.78
 2020-12-03    2777.78
 2020-12-04    2777.78
 2020-12-05    2777.78
 2020-12-06    2777.78
 2020-12-07    2777.78
 2020-12-08    2777.78
 2020-12-09    2777.78
 2020-12-10    2777.78
 2020-12-11    2777.78
 2020-12-12    2777.78
 2020-12-13    2777.78
 20